In [3]:
#!pip install -r requirements.txt

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os 
import numpy as np
import pinecone
from PyPDF2 import PdfFileReader, PdfReader # To read the PDF file
import langchain_core.documents.base as doc_base
#from langchain.embeddings import OpenAIEmbeddings # To convert the text chunks into tokens (or vectors or embeddings)
from langchain_google_genai import GoogleGenerativeAIEmbeddings # To convert the text chunks into tokens (or vectors or embeddings)
from langchain.vectorstores import Pinecone # To store the vectors in Pinecode Vector DB
from langchain_pinecone import PineconeVectorStore # To retive data from existing pinecone indexes
#from langchain.llms import OpenAI # The LLm model used
# from langchain.chat_models import ChatOpenAI
# import openapi
from langchain_google_genai import ChatGoogleGenerativeAI # The LLm model used to build the application
import google.generativeai as genai # Google's genai library to configure Gemini 

In [2]:
# load the API keys from .evn and configure Google GenAI

from dotenv import load_dotenv
print(load_dotenv())
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

True


In [3]:
# Read the document

def read_pdf_files(directory):
    """
    Function to load all PDF files in the mentioned dierectory. Groups the text by document and converts them to langchain_core.documents format.
    """
    texts = []  # List to store text content of all PDF files
    docs = [] # List to store the converted langchain_core.documents 
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):  # Check if the file is a PDF
            file_path = os.path.join(directory, filename)  # Get the full file path
            with open(file_path, 'rb') as file:  # Open the PDF file in binary mode
                pdf_reader = PdfReader(file)  # Create a PdfFileReader object
                num_pages = len(pdf_reader.pages)   # Get the number of pages in the PDF
                pdf_text = ''  # Variable to store text content of the PDF
                # Iterate over each page and extract text
                for page_num in range(num_pages):
                    page = pdf_reader.pages[page_num]  # Get the current page
                    pdf_text += page.extract_text()  # Extract text from the page and append to pdf_text
                texts.append(pdf_text)  # Append the text content of the PDF to the list
    
    # Convert the texts to langchain_core.documents format
    for doc_text in texts:
        docs.append(doc_base.Document(doc_text))
    return docs

docs= read_pdf_files(r"resources\input_files")
print(len(docs))
docs[:5]

24


[Document(page_content="The following te xts outlines Colorado's driving regulations, including requirements for \nobtaining a driver's license, notifying the DMV of changes, and details about written and \ndriving tests. It also provides guidance on maneuvers and behaviours  evaluated during the \ndriving skills test, emphasizing adherence to safety and legal driving practices.  \nAnyone who operates a motor vehicle, motor -driven cycle or moped on Colorado’s public \nstreets and highways must be 16 or older and have a valid driver license. If you are a resident of \nColorado (for example if you own or operate a business in Colorado or h ave resided within the \nstate continuously for 90 days or have gainful employment within this state), you must get a \nColorado driver license within 30 days of becoming a resident unless you are serving on active \nduty in the U.S. military, the dependent of a n active duty military service member or residing in \nColorado for the principal purpose 

Loaded all the 24 documents generated from the handbook.

In [4]:
# Initialze the embedder to create embeddings for the chunked documents

#embedder= OpenAIEmbeddings(api_key=os.environ['OPENAPI_API_KEY'])
embedder= GoogleGenerativeAIEmbeddings(model="models/embedding-001")
len(embedder.embed_query("Testing the length of embedded vector to be used in initializing the vector db?"))

768

In [5]:
# Initialize and store the vector embeddings of the chunked audio into Pinecone

pinecone.Pinecone(api_key= os.environ['PINECONE_API_KEY'], environment= os.environ['PINECONE_ENV']) # Initialize the pinecone with API Key 
index= Pinecone.from_documents(docs, embedder, index_name= os.environ['PINECONE_INDEX_NAME']) # Convert the chunked docs to vectors and stores them in Pinecone

In [7]:
# Create a funtion to return top matching vectors for the query based on Cosine Similarity

def pinecone_top_matching_vects(query, top_size= 2):
    top_matching_vects= index.similarity_search(query, k= top_size)
    return top_matching_vects

# Tesitng
for i in pinecone_top_matching_vects("Can I take the free right even if the signal is red?", top_size= 3):
    print(i.page_content)

The Following text provides comprehensive information about traffic controls, including 
traffic signals, traffic signs, and pavement markings. It explains the meaning of different 
traffic signal lights and how to respond to them, such as steady red, yellow, and green 
lights, as well as green, red, and yellow arrows. It also covers the significance of flashing 
lights and HAWK signals.  Furthermore, it details various types of traffic signs, such as 
regulatory signs, speed limit signs, stop signs, yield signs, and warning signs. It explains 
the importance of each sign and how drivers should respond to them to ensure safety on 
the road. Ad ditionally, it includes information about railroad crossings, work zone signs, 
guide signs, service/recreation signs, route signs, bicycle and pedestrian crossing signs, 
and disabled parking signs.  
TRAFFIC CONTROLS Traffic controls include traffic signals, traffic signs, and pavement 
markings. Traffic control also can be provided by law enfo